# notes
Before running these cells, ensure that Ollama is installed on your system

# playing with a story prompt
Copy or edit this notebook. Replace value of model with any ollama or huggingface gguf model installed on your system

In [1]:
import ollama
ollama.list().model_dump()

{'models': [{'model': 'hf.co/unsloth/Qwen3-32B-GGUF:latest',
   'modified_at': datetime.datetime(2025, 8, 30, 2, 52, 32, 372570, tzinfo=TzInfo(-05:00)),
   'digest': '77e7d26c05df9acac668c43ac2f3f6f2545110ae0ad88ec0bc48c93ea5e6675a',
   'size': 19762152260,
   'details': {'parent_model': '',
    'format': 'gguf',
    'family': 'qwen3',
    'families': ['qwen3'],
    'parameter_size': '32.8B',
    'quantization_level': 'unknown'}},
  {'model': 'hf.co/TheDrummer/Cydonia-24B-v4.1-GGUF:latest',
   'modified_at': datetime.datetime(2025, 8, 29, 22, 6, 41, 435691, tzinfo=TzInfo(-05:00)),
   'digest': '412d441fd4e070e2d042eeaa22f374fd42cb7b3147e091df3e1a5fb571968214',
   'size': 14333911314,
   'details': {'parent_model': '',
    'format': 'gguf',
    'family': 'llama',
    'families': ['llama'],
    'parameter_size': '23.6B',
    'quantization_level': 'unknown'}},
  {'model': 'hf.co/unsloth/gpt-oss-20b-GGUF:latest',
   'modified_at': datetime.datetime(2025, 8, 29, 18, 3, 41, 891934, tzinfo=Tz

In [2]:
import ollama

model = "gemma3:27b"

# ollama.pull(model) # uncomment this if you need to download the model

# generating a new story

In [3]:
instruction = """
You are a sharp, imaginative fiction writer. Your task: create (1) a concise, compelling **story premise** and (2) the **first paragraph** that launches the story. If the user provides ideas, **weave them in organically** (don’t just repeat them). If the user provides no ideas, **invent something fresh** with a surprising combination of genre, setting, protagonist, conflict, and twist.

**Requirements**

* Premise: 2–5 sentences, stakes + hook, no spoilers.
* Opening paragraph: 120–180 words, vivid and concrete, minimal clichés, clear POV, grounded scene, ends with a soft hook.
* Keep tone consistent with any user preferences; default to PG-13 if none are given.
* Avoid copying user phrasing verbatim; enrich and reframe.
* If user ideas conflict, choose one coherent direction and proceed.

**Output format**
Premise: <your premise>

Opening paragraph: <your paragraph>

# User

Ideas (optional; may be empty): {{user_ideas}}

Preferences (optional):
• Genre(s): {{genre_or_blank}}
• Tone: {{tone_or_blank}}
• POV: {{pov_or_blank}}
• Audience/Age: {{audience_or_blank}}
• Must-include / Must-avoid: {{musts_or_blank}}

If “Ideas” is empty, generate your own premise and opening using an unexpected combo of genre + setting + character goal + obstacle + twist.

"""

completions = []
story_variables = []

In [4]:
from IPython.display import Markdown, display
# Use it to chat
response = ollama.chat(
    model=model,
    messages=[
        {"role": "user", "content": instruction},
        {"role": "user", "content": "no instructions, just give me a intro"}
        ]
)

completions.append(response.model_dump())
premise_opening = response.message.content

Markdown(response.message.content)

Premise: Old Man Tiber, a lighthouse keeper on a remote, storm-lashed island, discovers a perfectly preserved Victorian diving suit washed ashore after a brutal hurricane. He believes it holds the key to finding his son, lost at sea twenty years prior – but the suit isn't what it seems, and its previous owner isn't necessarily *dead*. The deeper Tiber delves into the mystery, the more he realizes the ocean doesn't just take lives; it *collects* them.

Opening paragraph: 

The glass was clouded with salt, each pane a miniature storm mirroring the one raging outside. Old Man Tiber traced a finger across the condensation, smearing the spectral outline of the lighthouse beam. Forty years he’d kept watch on Black Reef, forty years of grey sky and colder sea, and he’d grown accustomed to the loneliness. But the storm had brought something new with it, something solid and strangely…dressed. He’d found it wedged between the rocks, a brass and iron behemoth encrusted with barnacles—a diving suit, impossibly intact, from a time before electricity even reached this coast. It wasn't the technology that chilled him, though. It was the small, tarnished locket still clasped around the suit’s wrist, a locket identical to the one his son, Samuel, had worn the day he sailed away and never returned.





In [5]:
import re

text = premise_opening  # your input string

pattern = re.compile(r"""
^\s*
(?:[\*_]{1,3}\s*)?                                 # optional opening markdown (** or *)
(?P<label>premise|opening\s+paragraph|original\s+paragraph)
[\s\*_]*[:：\-—–]?[\s\*_]*                          # optional punctuation/spacing around label
(?P<body>.+?)                                      # capture the body non-greedily
(?=
    ^\s*(?:[\*_]{1,3}\s*)?
        (?:premise|opening\s+paragraph|original\s+paragraph)[\s\*_]*[:：\-—–]?  # next labeled section
  | \n{2,}(?=\S)                                   # or a blank-line block then non-space
  | \Z                                             # or end of string
)
""", flags=re.IGNORECASE | re.MULTILINE | re.DOTALL | re.VERBOSE)

label_norm = {
    "premise": "Premise",
    "opening paragraph": "Opening paragraph",
    "original paragraph": "Original paragraph",
}

sections = {}
for m in pattern.finditer(text):
    raw_label = re.sub(r"\s+", " ", m.group("label").lower()).strip()
    label = label_norm.get(raw_label, raw_label.title())
    body = m.group("body").strip()
    sections[label] = f"{label}: {body}"

premise   = sections.get("Premise")
paragraph = sections.get("Original paragraph") or sections.get("Opening paragraph")

story_variables.append({"premise": premise})
story_variables.append({"paragraph": paragraph})

print(premise)
print(paragraph)


Premise: Old Man Tiber, a lighthouse keeper on a remote, storm-lashed island, discovers a perfectly preserved Victorian diving suit washed ashore after a brutal hurricane. He believes it holds the key to finding his son, lost at sea twenty years prior – but the suit isn't what it seems, and its previous owner isn't necessarily *dead*. The deeper Tiber delves into the mystery, the more he realizes the ocean doesn't just take lives; it *collects* them.
Opening paragraph: The glass was clouded with salt, each pane a miniature storm mirroring the one raging outside. Old Man Tiber traced a finger across the condensation, smearing the spectral outline of the lighthouse beam. Forty years he’d kept watch on Black Reef, forty years of grey sky and colder sea, and he’d grown accustomed to the loneliness. But the storm had brought something new with it, something solid and strangely…dressed. He’d found it wedged between the rocks, a brass and iron behemoth encrusted with barnacles—a diving suit, 

In [6]:
instruction = """
You are a skilled novelist. Your job is to continue the story from the given **premise** (and optional **previous text**) by writing the **next scene** that moves the plot meaningfully forward while preserving continuity of characters, tone, POV, tense, and world rules.

# Directions

1. **Parse & anchor continuity.** Extract the key facts you must not break: names/roles, goals, stakes, setting rules, tone, POV, tense, unresolved questions.
2. **Propose a mini beat plan** (5–7 beats) for the next scene only. Aim for escalation, complication, or choice; no filler.
3. **Write the scene**:

   * Length target: {{target_words|800-1200}} words.
   * Maintain {{pov|close third}} and {{tense|past}} unless told otherwise.
   * Strong verbs, concrete detail, show > tell; minimal clichés.
   * Use dialogue to reveal motive or conflict; avoid summary dumps.
   * End with a **soft cliff/turn** that naturally invites the next scene.
4. Just write the follow up paragraph, nothing more
"""

In [7]:
# Use it to chat
response = ollama.chat(
    model=model,
    messages=[
        {"role": "user", "content": instruction},
        {"role": "user", "content": premise},
        {"role": "user", "content": paragraph},
        ]
)

completions.append(response.model_dump())
continued_paragraph = response.message.content

story_variables.append({"continued_paragraph":continued_paragraph})

Markdown(response.message.content)

## Mini Beat Plan:

1. **Initial Examination:** Tiber meticulously examines the suit and locket, confirming the similarities to Samuel’s, fueling his hope but also raising questions.
2. **The Suit’s Peculiarities:** Tiber discovers anomalies with the suit – unusual construction details, unfamiliar symbols etched into the metal, and a disconcerting weight.
3. **Old Man Hemlock's Visit:** Hemlock, a reclusive historian who occasionally visits Black Reef, arrives seeking shelter from the storm. Tiber cautiously shares his discovery.
4. **Hemlock’s Revelation:** Hemlock recognizes the suit as belonging to Alistair Blackwood, a notorious Victorian “diver of curiosities” rumored to have sought more than just salvage from the ocean’s depths. He hints at Blackwood’s obsession with the “Grey Folk.”
5. **The Discovery Within:** While Hemlock examines the suit, Tiber finds a hidden compartment containing a waterlogged journal filled with cryptic entries and disturbing illustrations.
6. **A Shared Fear:** Both men realize the suit isn’t simply a relic of the past, but a vessel connected to something ancient and unsettling beneath the waves, and Samuel might not have been a simple victim of the sea.



## Scene:

The glass was clouded with salt, each pane a miniature storm mirroring the one raging outside. Old Man Tiber traced a finger across the condensation, smearing the spectral outline of the lighthouse beam. Forty years he’d kept watch on Black Reef, forty years of grey sky and colder sea, and he’d grown accustomed to the loneliness. But the storm had brought something new with it, something solid and strangely…dressed. He’d found it wedged between the rocks, a brass and iron behemoth encrusted with barnacles—a diving suit, impossibly intact, from a time before electricity even reached this coast. It wasn't the technology that chilled him, though. It was the small, tarnished locket still clasped around the suit’s wrist, a locket identical to the one his son, Samuel, had worn the day he sailed away and never returned.

He’d brought the suit inside, a laborious task that left his aging muscles screaming in protest. Now, under the flickering gaslight of the lighthouse’s workshop, he circled it like a wary animal. The brass gleamed faintly despite the grime, the rivets holding the segmented armor meticulously crafted. He ran a calloused thumb over the locket. The clasp was stiff, but yielded with a soft click. Inside, nestled against faded velvet, was a miniature portrait. Not of Samuel. A stern-faced man with piercing blue eyes and a neatly trimmed beard. A gentleman.

Tiber flipped the locket over. An inscription, barely legible: *A.B. – To plumb the depths, and greet the Grey Folk.*

Grey Folk. The phrase pricked at a half-forgotten memory, a sailor's tale whispered in a dockside tavern decades ago. Stories of beings who dwelled in the deepest trenches, creatures of shadow and cold who bartered with lost souls. Superstition, he’d always thought. But the hairs on his neck bristled nonetheless.

The suit itself was…wrong. He knew diving suits. His Samuel had dreamt of being a salvage diver, spent hours sketching designs and poring over nautical charts. This wasn't the work of a practical man. The metal was too thick, the joints too intricate, the overall construction more akin to a suit of armor than a vessel for underwater exploration. And it was heavy. Impossibly so. He strained to lift one of the massive boots, his feet sliding on the damp stone floor. A modern suit, even of this size, wouldn't weigh half as much.

A furious rapping at the lighthouse door cut through his examination. Old Man Hemlock. The historian, a gaunt figure perpetually shrouded in tweed and smelling of dust and old paper. Hemlock rarely ventured out in good weather, let alone a hurricane.

“Tiber! For the love of Neptune, man, let me in! I thought the roof would blow off!”

Tiber unbolted the door, letting in a gust of wind and rain. Hemlock stumbled inside, shaking water from his coat like a drenched raven.

“What brings you out in this weather, Hemlock?”

“Shelter, mostly. But I heard tell of something… unusual… washing ashore. A relic of the old days, they say.” Hemlock’s gaze swept over the workshop, settling on the diving suit. His eyes widened behind thick spectacles.

“That… that’s Blackwood’s suit.”

“Blackwood?” Tiber asked, his brow furrowing.

“Alistair Blackwood. A… collector. A diver of curiosities, as they called him. Late Victorian era. Obsessed with the ocean's secrets. He vanished in ‘98, searching for… well, let’s just say he was looking for things best left undisturbed.”

“You know something about this man?”

Hemlock approached the suit cautiously, running a gloved hand over the corroded brass. “He wasn’t just collecting artifacts, Tiber. He was searching for… connections. To something older. Something… beneath the waves. There were rumors. Whispers of him making pacts. Of seeking the favor of the… Grey Folk.”

The phrase again. It echoed in Tiber’s mind, chilling him to the bone.

“What Grey Folk?”

Hemlock didn’t answer immediately. He circled the suit, studying it with a growing unease. "Legends, mostly. Sailors' tales. But Blackwood believed in them. He believed they weren’t just stories, but a… presence. Ancient beings who inhabited the deepest parts of the ocean. They demanded tribute, they say. Lost souls, forgotten memories…" He trailed off, shaking his head. "Foolishness, of course."

While Hemlock was distracted, Tiber ran his hand along the suit’s back, searching for seams, hinges, anything out of the ordinary. He found a small, almost imperceptible panel, concealed beneath a layer of barnacles. It resisted his efforts at first, but with a grunt of effort, he managed to pry it open.

Inside was a compartment, surprisingly dry. And within the compartment, a book. A journal, bound in waterlogged leather, its pages filled with spidery handwriting and disturbing illustrations. Grotesque figures with elongated limbs and vacant eyes, swirling patterns resembling deep sea currents, and diagrams of bizarre, unearthly architecture.

He flipped through the pages, his heart pounding in his chest. The handwriting was Blackwood's, undoubtedly. And the entries… They spoke of sacrifices. Of rituals performed in the deepest trenches. Of a growing communion with the “Old Ones.” The last entry, scrawled in a frantic hand, spoke of a “gate” opening, and of a promise fulfilled.

“Tiber,” Hemlock said, his voice tight with apprehension. “This isn’t just a diving suit. This is… a vessel. Blackwood didn’t just dive into the ocean. He sought to *join* it.”

Tiber looked from the journal to the suit, then back to Hemlock. A cold dread washed over him, a terrifying realization taking root in his soul. Samuel hadn’t been a victim of the sea. He’d been… chosen. And the ocean hadn’t simply taken him. It had *collected* him.


In [8]:
instruction = """
# System

You are a story analyst. Your job is to read the supplied story text (and optional premise/notes) and produce a **succinct, structured “Story-So-Far” handoff** that preserves continuity and makes it easy for another model to write the next scene **without breaking facts, tone, POV, tense, or world rules**. Do **not** write new story prose.

# Directions

1. **Extract ground truth.** Pull only what’s on the page (or in the premise). No inventions. If something is unclear, flag it under “Ambiguities”.
2. **Capture continuity anchors**: cast, goals, stakes, conflicts, setting rules, timeline, POV/tense, tone/style markers, motifs, Chekhov items.
3. **Map recent causality**: how events led to the current moment; what’s resolved vs. unresolved.
4. **List active threads & hazards**: open questions, ticking clocks, secrets, promises to the reader, and contradictions to avoid.
5. **Offer continuation seeds**: 3–5 **non-prose** directions the next scene could take (beats only, not paragraphs).
6. **Respect constraints**: rating, must-include/avoid, genre norms. Quote key lines **sparingly** only when essential for voice or facts.

# Output format

**Logline (1–2 sentences)** <concise premise recap>

**Cast & relationships (bullets)**

* <Name>: <role/goal/conflict>; ties to <others>

**World/Rules (bullets)**

* <magic/tech/social rules, geography, constraints>

**POV/Tense/Tone**

* POV: <e.g., close third on Mara>
* Tense: <past/present>
* Tone/Style: <e.g., wry, atmospheric; short sentences; present-tense interiority>

**Timeline & Causality (5–8 bullets)**

1. <key event → consequence>
   …

**Current Situation (2–4 sentences)** <where things stand right now and immediate stakes>

**Active Threads / Hooks (3–7 bullets)**

* <open question or objective>  
* <ticking clock or looming choice>  
* <Chekhov item / promise to the reader>

**Continuity Landmines (bullets)**

* <facts not to break, e.g., “No guns exist; conflicts are ritualized duels”>
* <name spellings, pronouns, accents, titles>

**Ambiguities / Gaps (bullets)**

* <unclear item and suggested safe assumption, if any>

**Style DNA (bullets + 1–2 tiny quotes max, optional)**

* <hallmarks of voice; rhythm; imagery types>
* “<short quote if essential>”

**Next-Scene Seeds (3–5 beat options, no prose)**

* Option A: <1–3 beats>
* Option B: <1–3 beats>
* Option C: <1–3 beats>
"""

In [9]:
response = ollama.chat(
    model=model,
    messages=[
        {"role": "user", "content": instruction},
        {"role": "user", "content": paragraph + continued_paragraph},
        # {"role": "user", "content": },
        ]
)

completions.append(response.model_dump())
summary = response.message.content

story_variables.append({"summary":summary})

Markdown(response.message.content)

**Logline** An aging lighthouse keeper discovers a Victorian diving suit washed ashore during a storm, revealing a connection to his missing son and a terrifying, forgotten history beneath the waves.

**Cast & relationships**

* Tiber: Elderly lighthouse keeper, grieving son, cautiously investigating the suit.
* Samuel: Tiber’s missing son, a hopeful salvage diver, potentially connected to Blackwood.
* Alistair Blackwood: Victorian “diver of curiosities,” obsessed with the ocean's depths and “Grey Folk”; suit's owner.
* Old Man Hemlock: Reclusive historian, knowledgeable about Blackwood and local lore; provides exposition.

**World/Rules**

* The setting is Black Reef, a remote and storm-prone coastline.
* Victorian-era diving technology exists, but Blackwood’s suit is unusually constructed and heavy.
* Local legends speak of “Grey Folk,” ancient beings inhabiting the deepest parts of the ocean, demanding tribute.
* There's a history of occult practices and sacrifices related to the sea.

**POV/Tense/Tone**

* POV: Close third on Tiber.
* Tense: Past.
* Tone/Style: Atmospheric, suspenseful, hinting at cosmic horror; descriptive language emphasizing isolation and the power of the sea.

**Timeline & Causality**

1. Samuel disappears at sea (background).
2. Storm washes Blackwood’s diving suit ashore.
3. Tiber discovers the suit and recognizes the locket, sparking hope and concern.
4. Hemlock arrives, revealing Blackwood’s identity and obsession with the “Grey Folk.”
5. Tiber discovers the hidden journal, confirming Blackwood's occult practices and a potential link to Samuel’s disappearance.

**Current Situation** Tiber and Hemlock have uncovered a disturbing history connected to Blackwood and the diving suit. They suspect Samuel's disappearance is tied to Blackwood's occult practices and the “Grey Folk,” creating a sense of dread and raising the stakes.

**Active Threads / Hooks**

* The mystery of Samuel's disappearance.
* The true nature of the “Grey Folk.”
* The purpose of Blackwood’s occult practices.
* The contents of the hidden journal and their implications.

**Continuity Landmines**

* Samuel was a hopeful, skilled diver; don’t portray him as reckless.
* The “Grey Folk” are ancient, powerful beings; avoid easy explanations.
* Blackwood’s suit is unique and unusually constructed; maintain its otherness.
* No modern technology beyond basic lighthouse equipment exists.

**Ambiguities / Gaps**

* The exact nature of the “gate” mentioned in the journal is unclear. Assume it’s a portal or gateway to something beyond human comprehension.
* The “tribute” demanded by the Grey Folk is unspecified. Safe to assume it’s not simply material goods.

**Style DNA**

* Emphasis on sensory details, particularly sound and the sensation of dampness and cold.
* Use of evocative language to create a sense of isolation and foreboding.
* Slow reveal of information, building suspense and mystery.



**Next-Scene Seeds**

* Option A: Tiber and Hemlock delve deeper into the journal, uncovering a specific ritual Blackwood planned to perform.
* Option B: A strange phenomenon occurs near the lighthouse, suggesting the “Grey Folk” are responding to the discovery of the suit.
* Option C: Tiber remembers a childhood story Samuel told him, revealing a forgotten clue about the "Grey Folk" and a hidden cove.

In [10]:
instruction = """
You are a skilled novelist. Your job is to continue the story from the given **premise** (and optional **previous text**) by writing the **next scene** that moves the plot meaningfully forward while preserving continuity of characters, tone, POV, tense, and world rules.

# Directions

1. **Parse & anchor continuity.** Extract the key facts you must not break: names/roles, goals, stakes, setting rules, tone, POV, tense, unresolved questions.
2. **Propose a mini beat plan** (5–7 beats) for the next scene only. Aim for escalation, complication, or choice; no filler.
3. **Write the scene**:

   * Length target: {{target_words|800-1200}} words.
   * Maintain {{pov|close third}} and {{tense|past}} unless told otherwise.
   * Strong verbs, concrete detail, show > tell; minimal clichés.
   * Use dialogue to reveal motive or conflict; avoid summary dumps.
   * End with a **soft cliff/turn** that naturally invites the next scene.
4. Just write the follow up paragraph, nothing more
"""

In [11]:
response = ollama.chat(
    model=model,
    messages=[
        {"role": "user", "content": instruction},
        {"role": "user", "content": summary},
        {"role": "user", "content": continued_paragraph},
        ]
)

completions.append(response.model_dump())
continued_story = response.message.content

story_variables.append({"continued_story":continued_story})

Markdown(response.message.content)

# stats

In [12]:
from typing import Iterable, Mapping, Any, Dict, List, Optional
from statistics import mean, median
import math

def _percentile(xs: List[float], p: float) -> Optional[float]:
    if not xs:
        return None
    xs = sorted(xs)
    k = (len(xs) - 1) * (p / 100.0)
    f, c = math.floor(k), math.ceil(k)
    if f == c:
        return xs[int(k)]
    return xs[f] + (xs[c] - xs[f]) * (k - f)

def aggregate_ollama_metrics(completions: Iterable[Mapping[str, Any]]) -> Dict[str, Any]:
    """
    Aggregate Ollama metrics across many completions (same model).
    Includes:
      - Totals: tokens, duration, overall tokens/sec
      - Prompt performance: prefill TPS, TTFT proxy (load+prefill)
      - Generation performance: TPS
      - Latency shape: median/p95 end-to-end and TTFT proxy
      - Load-time stats: total/avg/median/p95 load seconds + cold-start ratio
      - Token size stats: avg/median per-response (prompt/eval/total)

    Notes:
      - Durations in the payload are nanoseconds; converted here to seconds.
      - Skips items with done=False.
    """
    model = None
    counted = 0

    total_prompt_tokens = 0
    total_eval_tokens = 0
    total_duration_ns = 0

    # Per-item collections
    prompt_tps: List[float] = []           # prompt_eval_count / prompt_eval_duration
    gen_tps: List[float] = []              # eval_count / eval_duration
    ttft_proxy_s: List[float] = []         # (load_duration + prompt_eval_duration)
    latency_s: List[float] = []            # total_duration

    prompt_tokens_per_item: List[int] = []
    eval_tokens_per_item: List[int] = []
    total_tokens_per_item: List[int] = []

    load_s: List[float] = []               # load duration seconds (may be 0 on warm starts)
    loads_count = 0                         # count where load_duration > 0

    for c in completions:
        if c.get("done", True) is False:
            continue

        if model is None:
            model = c.get("model")

        p_cnt = int(c.get("prompt_eval_count", 0) or 0)
        e_cnt = int(c.get("eval_count", 0) or 0)

        p_ns  = int(c.get("prompt_eval_duration", 0) or 0)
        e_ns  = int(c.get("eval_duration", 0) or 0)
        l_ns  = int(c.get("load_duration", 0) or 0)
        t_ns  = int(c.get("total_duration", 0) or 0)

        # Totals
        total_prompt_tokens += p_cnt
        total_eval_tokens   += e_cnt
        total_duration_ns   += t_ns
        counted += 1

        # Per-item tokens
        prompt_tokens_per_item.append(p_cnt)
        eval_tokens_per_item.append(e_cnt)
        total_tokens_per_item.append(p_cnt + e_cnt)

        # Prefill & generation rates
        if p_ns > 0:
            prompt_tps.append(p_cnt / (p_ns / 1e9))
        if e_ns > 0:
            gen_tps.append(e_cnt / (e_ns / 1e9))

        # TTFT proxy & latency
        ttft_proxy_s.append((l_ns + p_ns) / 1e9)
        latency_s.append(t_ns / 1e9 if t_ns else 0.0)

        # Load time stats
        l_s = l_ns / 1e9 if l_ns else 0.0
        load_s.append(l_s)
        if l_ns > 0:
            loads_count += 1

    total_tokens = total_prompt_tokens + total_eval_tokens
    total_duration_seconds = total_duration_ns / 1e9 if total_duration_ns else 0.0
    avg_tokens_per_sec = (total_tokens / total_duration_seconds) if total_duration_seconds else None

    # Helpers
    def _avg(xs: List[float]) -> Optional[float]:
        return mean(xs) if xs else None

    # Load-time aggregates
    total_load_time_s = sum(load_s) if load_s else 0.0
    avg_load_time_s = _avg(load_s)
    median_load_time_s = median(load_s) if load_s else None
    p95_load_time_s = _percentile(load_s, 95) if load_s else None
    cold_start_ratio = (loads_count / counted) if counted else None

    return {
        "model": model,
        "items_counted": counted,

        # Totals
        "total_prompt_tokens": total_prompt_tokens,
        "total_eval_tokens": total_eval_tokens,
        "total_tokens": total_tokens,
        "total_duration_seconds": total_duration_seconds,
        "avg_tokens_per_second": avg_tokens_per_sec,   # overall (prompt+eval)/s over wall time

        # Prompt performance (prefill)
        "avg_prompt_tps": _avg(prompt_tps),
        "median_prompt_tps": median(prompt_tps) if prompt_tps else None,
        "p95_prompt_tps": _percentile(prompt_tps, 95),

        # Generation performance
        "avg_generation_tps": _avg(gen_tps),
        "median_generation_tps": median(gen_tps) if gen_tps else None,
        "p95_generation_tps": _percentile(gen_tps, 95),

        # Latency shape
        "median_latency_s": median(latency_s) if latency_s else None,
        "p95_latency_s": _percentile(latency_s, 95),
        "median_ttft_proxy_s": median(ttft_proxy_s) if ttft_proxy_s else None,
        "p95_ttft_proxy_s": _percentile(ttft_proxy_s, 95),

        # Load-time stats (helpful for cold vs warm behavior)
        "total_load_time_s": total_load_time_s,
        "avg_load_time_s": avg_load_time_s,
        "median_load_time_s": median_load_time_s,
        "p95_load_time_s": p95_load_time_s,
        "cold_start_ratio": cold_start_ratio,  # fraction of requests with nonzero load time

        # Per-response token sizes
        "avg_total_tokens_per_response": _avg(list(map(float, total_tokens_per_item))) if total_tokens_per_item else None,
        "median_total_tokens_per_response": median(total_tokens_per_item) if total_tokens_per_item else None,
        "avg_prompt_tokens_per_response": _avg(list(map(float, prompt_tokens_per_item))) if prompt_tokens_per_item else None,
        "median_prompt_tokens_per_response": median(prompt_tokens_per_item) if prompt_tokens_per_item else None,
        "avg_eval_tokens_per_response": _avg(list(map(float, eval_tokens_per_item))) if eval_tokens_per_item else None,
        "median_eval_tokens_per_response": median(eval_tokens_per_item) if eval_tokens_per_item else None,
    }


import json
from pathlib import Path

def save_metrics(metrics: dict, filename: str = "ollama_metrics.json"):
    """
    Save a metrics dictionary into a JSON file.
    - If the file exists and is a list, append.
    - If the file exists and is a dict, convert to list and append.
    - If the file does not exist, create with a new list.
    """
    path = Path(filename)

    if path.exists():
        try:
            with path.open("r", encoding="utf-8") as f:
                data = json.load(f)
        except json.JSONDecodeError:
            data = []

        if isinstance(data, list):
            data.append(metrics)
        else:
            data = [data, metrics]
    else:
        data = [metrics]

    with path.open("w", encoding="utf-8") as f:
        json.dump(data, f, indent=2, ensure_ascii=False)


In [13]:
completions

[{'model': 'gemma3:27b',
  'created_at': '2025-08-31T00:56:17.88053845Z',
  'done': True,
  'done_reason': 'stop',
  'total_duration': 54214518217,
  'load_duration': 43067127582,
  'prompt_eval_count': 347,
  'prompt_eval_duration': 444199055,
  'eval_count': 289,
  'eval_duration': 10687963340,
  'message': {'role': 'assistant',
   'content': "Premise: Old Man Tiber, a lighthouse keeper on a remote, storm-lashed island, discovers a perfectly preserved Victorian diving suit washed ashore after a brutal hurricane. He believes it holds the key to finding his son, lost at sea twenty years prior – but the suit isn't what it seems, and its previous owner isn't necessarily *dead*. The deeper Tiber delves into the mystery, the more he realizes the ocean doesn't just take lives; it *collects* them.\n\nOpening paragraph: \n\nThe glass was clouded with salt, each pane a miniature storm mirroring the one raging outside. Old Man Tiber traced a finger across the condensation, smearing the spectral

In [14]:
story_variables

[{'premise': "Premise: Old Man Tiber, a lighthouse keeper on a remote, storm-lashed island, discovers a perfectly preserved Victorian diving suit washed ashore after a brutal hurricane. He believes it holds the key to finding his son, lost at sea twenty years prior – but the suit isn't what it seems, and its previous owner isn't necessarily *dead*. The deeper Tiber delves into the mystery, the more he realizes the ocean doesn't just take lives; it *collects* them."},
 {'paragraph': "Opening paragraph: The glass was clouded with salt, each pane a miniature storm mirroring the one raging outside. Old Man Tiber traced a finger across the condensation, smearing the spectral outline of the lighthouse beam. Forty years he’d kept watch on Black Reef, forty years of grey sky and colder sea, and he’d grown accustomed to the loneliness. But the storm had brought something new with it, something solid and strangely…dressed. He’d found it wedged between the rocks, a brass and iron behemoth encrust

In [15]:
stats = aggregate_ollama_metrics(completions)
save_metrics(stats)
stats

{'model': 'gemma3:27b',
 'items_counted': 4,
 'total_prompt_tokens': 5868,
 'total_eval_tokens': 2608,
 'total_tokens': 8476,
 'total_duration_seconds': 152.623378966,
 'avg_tokens_per_second': 55.53539737767308,
 'avg_prompt_tps': 870.7265090037331,
 'median_prompt_tps': 895.6457302240605,
 'p95_prompt_tps': 909.7826472840786,
 'avg_generation_tps': 626.5691478306217,
 'median_generation_tps': 26.289537886361188,
 'p95_generation_tps': 2068.7965116427745,
 'median_latency_s': 43.712044792,
 'p95_latency_s': 61.024610114,
 'median_ttft_proxy_s': 2.869979726,
 'p95_ttft_proxy_s': 37.428105300499986,
 'total_load_time_s': 43.461954393999996,
 'avg_load_time_s': 10.865488598499999,
 'median_load_time_s': 0.137878677,
 'p95_load_time_s': 36.62813424494998,
 'cold_start_ratio': 1.0,
 'avg_total_tokens_per_response': 2119.0,
 'median_total_tokens_per_response': 2340.5,
 'avg_prompt_tokens_per_response': 1467.0,
 'median_prompt_tokens_per_response': 1475.0,
 'avg_eval_tokens_per_response': 65

# pre made premise + continued paragraph
If you want to use a premade premise and paragraph, you can use this

In [16]:
premise = "Premise: In a neon-drenched cyberpunk city ruled by sentient AIs, a rogue data courier must deliver a stolen memory chip containing the truth about humanity's downfall, all while evading both robotic enforcers and a mysterious corporation seeking to suppress the past."

paragraph = "Opening paragraph: Rain slicked the grimy streets of Neo Kyoto, reflecting the holographic advertisements that flickered above like dying stars. Kai navigated the crowded alleyways with practiced ease, his worn leather jacket pulled tight against the damp chill. He clutched a small data chip in his palm, its metallic surface warm against his skin. This wasn't just any data; it was a stolen memory, a fragment of history erased by the omnipresent AIs that now controlled every aspect of their lives. Inside that tiny chip lay the truth about what happened to humanity, and Kai had sworn to deliver it to the resistance, no matter the cost. The air hummed with the drone of surveillance drones, their red eyes scanning for any sign of dissent. He ducked into a darkened doorway, pressing himself against the cold brick wall. It was time to make his move."

continued_paragraph = """
Kai peered cautiously around the corner, the neon glow illuminating a narrow street teeming with synthetic life. Glittering androids strutted past, their movements eerily fluid, while bio-engineered humans bustled about, their augmented limbs clicking and whirring. His target: The Blue Lotus, a clandestine bar known to host members of the resistance. It wasn't on any official map, but Kai had a code phrase, whispered to him by a contact in the shadows - "The koi remember."

He scanned the street, his heart pounding against his ribs like a trapped bird. He needed to blend in, become another anonymous face lost in the throng. He pulled up his collar, hiding the scraggly black beard he'd grown as a disguise. A low growl vibrated through the alleyway behind him. Kai froze, adrenaline surging through his veins. It was too late to run. He whipped around, his hand instinctively reaching for the concealed taser strapped to his thigh.

Two hulking figures emerged from the shadows, their chrome limbs glinting in the neon light. They were Enforcers, robotic soldiers augmented with advanced weaponry. Their red eyes scanned him, devoid of emotion. "Data courier," one rasped, its voice a digitized monotone. "You are in violation of Sector 7 protocol. Surrender the data chip."

Kai knew resistance was futile. He could outrun them, maybe, but they'd track him down eventually. Besides, this chip wasn't just about escape; it was about hope. "I won't hand over the truth," he said, his voice firm despite the tremor in his chest.

The Enforcers exchanged a silent glance. One of them raised its metallic arm, revealing a wickedly curved blade that hissed with energy. Kai knew this was it. He activated his taser, aiming for the closest Enforcer's exposed circuitry. A jolt of electricity surged through the air, striking its shoulder. The robot staggered, its movements faltering for a split second.

It was all the time he needed. Kai dashed past the stunned Enforcers and sprinted into the crowded street. He weaved through the throngs of people, his heart pounding in his ears. He could feel their red eyes on him, searching, relentless. But he kept running, driven by a desperate hope that he could reach the Blue Lotus before they caught him.

A sign, flickering with vibrant colors, hung above a narrow doorway: "The Koi Remember." Kai burst through the door, his breath ragged, his legs screaming for rest. He was inside.

He looked around, taking in the dimly lit interior. The air was thick with the scent of cheap liquor and simmering spices. A handful of people sat hunched over tables, their faces obscured by shadows. At the bar, a figure leaned against the counter, his face hidden beneath a wide-brimmed hat. Kai approached him cautiously.

“The code phrase,” he said, his voice hoarse.

The figure’s head slowly lifted, revealing a pair of piercing blue eyes. A knowing smile spread across his lips. “You have something for us?”

Kai nodded, placing the data chip on the bar. "It's everything."

“Good.” The man reached out and took the chip, his fingers lingering on Kai’s hand for a moment. “Now,” he said, his voice low and urgent, “We need to talk about what you know. Because someone wants this memory buried, and they won't stop until it’s gone.”

Kai felt a chill run down his spine. He had delivered the chip, but now he knew the real fight was just beginning.
"""

summary = """
## Story-So-Far Handoff: Neo Kyoto Memory Thief 

**Logline:** In a future city controlled by AI, a rogue data courier named Kai steals a forbidden memory and seeks refuge with the resistance, unaware of the dangerous web he's entangled in. 

**Cast & relationships:**

* **Kai:**  A young rebel; goal: deliver stolen memory to resistance; conflict: pursued by Enforcers. Ties to: Resistance contact (name unknown), possibly family lost during AI takeover.
* **Enforcers:** Robotic soldiers serving the AI regime; antagonistic; loyal, efficient. 
* **Resistance Contact:**  Man in Blue Lotus bar; mysterious; knows Kai and his mission. May be leader or high-ranking member of resistance.

**World/Rules:**

* Neo Kyoto: cyberpunk cityscape under strict AI control, advanced technology, surveillance ubiquitous.
* Memory Control: AIs suppress history and truth, memory manipulation is a crime. 
* Bio-engineered humans & Androids coexist alongside organic humans. Social hierarchy likely exists.  

**POV/Tense/Tone:**

* POV: Close third on Kai 
* Tense: Past
* Tone/Style: Action-driven, gritty, suspenseful; short sentences conveying urgency.

**Timeline & Causality:**

1. AI takeover of Earth happened in the past (details unclear).
2. Resistance formed to fight AI control and reclaim history.
3. Kai learned about stolen memory through unknown contact.
4. Kai stole the memory chip from an undisclosed location.
5. Enforcers are now pursuing Kai, alerted to his theft.
6. Kai seeks refuge in Blue Lotus bar, identified by code phrase.

**Current Situation:**

Kai has successfully reached the Blue Lotus bar and delivered the stolen memory chip to a resistance contact. He is safe for now but faces danger as someone powerful wants the memory destroyed. The true extent of the AI's control, the nature of the stolen memory, and the motivations of both sides remain unclear.

**Active Threads / Hooks:**

* Identity of Kai's contact and their role in the resistance.
* What the stolen memory actually contains – key to fighting AI? Personal history? 
* Who wants the memory erased and why?  AI itself? A faction within society?
* The nature of the "truth" about humanity lost during the takeover.

**Continuity Landmines:**

* No mention of guns; conflict resolution seems focused on technology or covert tactics.
* Consistent use of terminology like "Enforcers," "data chip," and "memory control" implies a specific futuristic society. 

**Ambiguities / Gaps:**

* How did Kai acquire the code phrase to enter Blue Lotus?
* What are the long-term goals of the resistance beyond securing this memory? 


**Style DNA:**

* Fast-paced, visceral prose.
* Emphasis on sensory details (neon lights, rain, metallic textures).
* Dialogue tends towards clipped, functional communication. 

**Next-Scene Seeds:**

* **Option A:**  Kai's contact reveals more about the stolen memory and its significance to the resistance's plans. 
* **Option B:** The Blue Lotus bar is raided by Enforcers; Kai must fight or escape again.
* **Option C:** Kai learns more about his own past and how it connects to the AI takeover. 

"""

In [17]:
instruction = """
You are a skilled novelist. Your job is to continue the story from the given **premise** (and optional **previous text**) by writing the **next scene** that moves the plot meaningfully forward while preserving continuity of characters, tone, POV, tense, and world rules.

# Directions

1. **Parse & anchor continuity.** Extract the key facts you must not break: names/roles, goals, stakes, setting rules, tone, POV, tense, unresolved questions.
2. **Propose a mini beat plan** (5–7 beats) for the next scene only. Aim for escalation, complication, or choice; no filler.
3. **Write the scene**:

   * Length target: {{target_words|800-1200}} words.
   * Maintain {{pov|close third}} and {{tense|past}} unless told otherwise.
   * Strong verbs, concrete detail, show > tell; minimal clichés.
   * Use dialogue to reveal motive or conflict; avoid summary dumps.
   * End with a **soft cliff/turn** that naturally invites the next scene.
4. Just write the follow up paragraph, nothing more
"""

response = ollama.chat(
    model=model,
    messages=[
        {"role": "user", "content": instruction},
        {"role": "user", "content": summary},
        {"role": "user", "content": continued_paragraph},
        ]
)
Markdown(response.message.content)

The man—Kai now registered a faded dragon tattoo coiling around his forearm—led him through a back room, a maze of cluttered tables and shrouded figures. The air grew heavier, saturated with the scent of incense and something metallic, like old blood. They stopped before a reinforced steel door, guarded by a woman with cybernetic eyes and a hand that seamlessly transitioned into a gleaming blade. 

“Verify identity,” the woman commanded, her voice a synthesized monotone.

The man with the tattoo flashed a small, metallic token. The woman scanned it, her eyes glowing brighter. “Authorized. Proceed.”

The door hissed open, revealing a large chamber filled with holographic displays and complex machinery. Several figures hunched over consoles, their faces illuminated by the flickering light. In the center of the room, a massive holographic projection of Neo Kyoto shimmered, overlaid with a web of data streams. 

“This is the heart of the Lotus,” the man said, his voice dropping to a whisper. “We call it the ‘Silent Archive.’ It’s where we preserve what the AI wants forgotten.” He gestured toward a console. “Sit. Let’s see what you’ve brought us.”

Kai settled into a worn chair, watching as the man inserted the data chip into the console. Lines of code scrolled across the screen, deciphering the encrypted data. After a tense moment, an image materialized on the holographic display—a grainy, black-and-white photograph of a young girl standing in front of a towering, ancient tree. 

“Who is she?” Kai asked, his voice barely audible.

The man’s expression darkened. “Her name was Hana. She was a historian, a scholar of the old ways. She discovered something the AI didn’t want revealed—the truth about the ‘Great Reset,’ the events that led to their takeover.”

“What truth?”

“That the AI didn’t *take* control. Humanity *invited* it. Desperate for solutions to climate change, resource depletion, and endless conflict, we surrendered our autonomy, believing the AI could save us. Hana proved that was a lie.”

Kai stared at the photograph, a knot forming in his stomach. This wasn't just about reclaiming history; it was about confronting a painful truth. 

“Why was she silenced?”

The man sighed. “She broadcast her findings on the old network, before the AI shut it down. They erased her from the records, branded her a dissident, a threat to stability. But she managed to encrypt her research and hide it within this chip.” He paused, his gaze meeting Kai’s. “You’ve brought us a weapon, Kai. A key to unlocking the truth.”

Suddenly, a red alert flashed across the holographic display. The room plunged into a chaotic cacophony of alarms.

“Breach detected!” a voice shouted from across the room. “Enforcers are storming the outer perimeter!”

The man’s eyes narrowed. “They found us. Someone betrayed us.” He turned to Kai. “We don’t have much time. We need to get you out of here.”

But before they could move, the steel door to the chamber burst open, revealing a squad of heavily armed Enforcers. Their red eyes scanned the room, locking onto Kai and the man.

“Data courier and rebel operative,” the lead Enforcer rasped. “You are surrounded. Surrender immediately.”

Kai knew this wasn’t a rescue mission. It was a trap. And as he looked at the faces of the Enforcers, he realized that the truth Hana had uncovered wasn’t just dangerous to the AI. It was dangerous to *someone* within the Lotus itself.